In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from datetime import date

import pandas as pd

In [ ]:
app = Flask(__name__)
db_name = 'database/member_lists.db'

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + db_name

app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True

# this variable, db, will be used for all SQLAlchemy commands
db = SQLAlchemy(app)

In [ ]:
class Member(db.Model): 
    __tablename__ = 'members_titles'
    id = db.Column(db.Integer, primary_key=True)
    title_id = db.Column(db.Integer)
    name_en = db.Column(db.String(255))
    name_ru = db.Column(db.String(255))
    max_ep = db.Column(db.Integer)
    from_usr = db.Column(db.String(100))
    to_usr = db.Column(db.String(100))

    def __init__(self, title_id, name_en, name_ru, max_ep, from_usr, to_usr):
        self.title_id = title_id
        self.name_en = name_en
        self.name_ru = name_ru
        self.max_ep = max_ep
        self.from_usr = from_usr
        self.to_usr = to_usr


In [ ]:
db.create_all()

In [76]:
# delete all data from the table
db.session.query(Member).delete() 
db.session.commit()

In [ ]:
# get data from SHIKIMORI
G_DRIVE_PATH = "G:\\My Drive\\ANIME_CHALLENGE"
from member_list import members
import yaml
from SHIKI_API import ShiNoAuth
a = ShiNoAuth()
trottle = 1
df_usr_dict = {}
for username in members:
        # read yaml file of member to list of ids
        # read from G_DRIVE
        # sleep(0.4)
        filename = f'{G_DRIVE_PATH}/{username}_list.yaml'
        with open(filename, 'r') as f:
            dl = yaml.load(f, Loader=yaml.FullLoader)
            # left only valid ids
            ids = [i for i in dl.values() if isinstance(i, int)]
        # get df w/ titles
        dfr = a.get_title_by_ids(ids, trottle)
        # name of user who assigned title
        dfr['from user'] = username
        # user who need to watch title
        dfr['to user'] = [key for key, val in dl.items() if isinstance(val, int)] 
        # df w/ all titles of member list
        df_usr_dict[username] = dfr
{i: list(df_usr_dict[i].values.tolist()) for i in members}

In [78]:
# write data to SQLAlchemy
for i in members:
    df = df_usr_dict[i]
    for index, row in df.iterrows():
        # print(row.to_dict()['id'])
        record = Member(title_id=row.to_dict()['id'], 
            name_en=row.to_dict()['name_en'], 
            name_ru=row.to_dict()['name_ru'], 
            max_ep=row.to_dict()['ep_max'], 
            from_usr=row.to_dict()['from user'], 
            to_usr=row.to_dict()['to user']
            )
        db.session.add(record)
db.session.commit()
    

In [83]:
pd.read_sql(f'SELECT * FROM members_titles WHERE "to_usr" LIKE "{members[-1]}%"',
            'sqlite:///' + db_name, 
            # index_col='id',
            )

,id,title_id,name_en,name_ru,max_ep,from_usr,to_usr
0,1,32995,Yuri!!! on Ice,Юри на льду,12,𝒜𝓁𝓁𝓎𝓇𝒾𝑒𝓁,Elaeagnus_1
1,2,33421,Hitori no Shita: The Outcast,Один из отвергнутых: Изгой,12,𝒜𝓁𝓁𝓎𝓇𝒾𝑒𝓁,Elaeagnus_2
2,3,1210,NHK ni Youkoso!,Добро пожаловать в NHK,24,𝒜𝓁𝓁𝓎𝓇𝒾𝑒𝓁,Elaeagnus_3
3,4,32093,Tanaka-kun wa Itsumo Kedaruge,Всегда вялый Танака-кун,12,𝒜𝓁𝓁𝓎𝓇𝒾𝑒𝓁,Elaeagnus_4
4,5,33487,Masamune-kun no Revenge,Месть Масамунэ!,12,𝒜𝓁𝓁𝓎𝓇𝒾𝑒𝓁,Elaeagnus_5
5,6,33926,Quanzhi Gaoshou,Аватар короля,12,𝒜𝓁𝓁𝓎𝓇𝒾𝑒𝓁,Elaeagnus_6
6,13,42249,Tokyo Revengers,Токийские мстители,24,AhresSsio,Elaeagnus_1
7,14,28907,"Gate: Jieitai Kanochi nite, Kaku Tatakaeri",Врата: Там бьются наши воины,12,AhresSsio,Elaeagnus_2
8,15,38759,Sewayaki Kitsune no Senko-san,Заботливая 800-летняя жена!,12,AhresSsio,Elaeagnus_3
9,16,48406,Re-Main,Ре-Мейн,12,AhresSsio,Elaeagnus_4
